In [1]:
# import dependecies 
from config import gkey

import requests 
import json

from pprint import pprint
import pandas as pd

In [2]:
# read movie list 
file = "pam/movies_list.csv"
movies_df = pd.read_csv(file, encoding='utf8')
movies_df.head()

,Number,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,76,2020-12-31,355,75000000.0,0.0,0.0
1,98,2020-12-31,Moonfall,150000000.0,0.0,0.0
2,10,2020-12-31,Hannibal the Conqueror,50000000.0,0.0,0.0
3,37,2020-09-30,Unhinged,29000000.0,0.0,0.0
4,32,2020-07-17,Tenet,224000000.0,0.0,0.0


In [3]:
# filter to 2018 movies
filtered_2018 = movies_df[(movies_df['Release Date'] > '2018-01-01') & (movies_df['Release Date'] < '2018-12-31')]
filtered_2018.tail()
# total 220

,Number,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
216,8,2018-01-19,Forever My Girl,3500000.0,16376066.0,16376066.0
217,69,2018-01-19,12 Strong,35000000.0,45819713.0,71118378.0
218,67,2018-01-12,The Commuter,40000000.0,36343858.0,101985431.0
219,57,2018-01-12,Proud Mary,30000000.0,20868638.0,21709539.0
220,54,2018-01-05,Insidious: The Last Key,10000000.0,67745330.0,172811971.0


In [4]:
# create list from movie column
movies = movies_df['Movie'].tolist()

In [5]:
# replace encoding issue 
encode_list = [movie.replace('â\x80\x99', "'").replace("â\x80\x94", "-").replace("Âº", "º").replace("Ã©", "e") for movie in movies]
encode_list

['355',
 'Moonfall',
 'Hannibal the Conqueror',
 'Unhinged',
 'Tenet',
 'Call of the Wild',
 'Army of the Dead',
 'Down Under Cover',
 'Rogue City',
 'Eli',
 'Reagan',
 'The Rhythm Section',
 'Midway',
 'Arctic Dogs',
 'Gisaengchung',
 'Ad Astra',
 'The Goldfinch',
 'PLAYMOBIL',
 'Blinded by the Light',
 'Once Upon a Timeâ\x80¦in Hollywood',
 'Crawl',
 'Spider-Man: Far From Home',
 'Anna',
 'Burn Your Maps',
 'Kursk',
 'Men in Black: International',
 'Shaft',
 'Late Night',
 'The Secret Life of Pets 2',
 'Dark Phoenix',
 'Rocketman',
 'Godzilla: King of the Monsters',
 'MA',
 'Aladdin',
 'BrightBurn',
 'John Wick: Chapter 3 - Parabellum',
 'The Sun is Also a Star',
 'The Professor and the Madman',
 'Pokemon: Detective Pikachu',
 'UglyDolls',
 'El Chicano',
 'Long Shot',
 'The Intruder',
 'Avengers: Endgame',
 'Living Dark: The Story of Ted the Caver',
 'Hellboy',
 'Shazam!',
 'Pet Sematary',
 'The Best of Enemies',
 'Dumbo',
 'Unplanned',
 'Us',
 'Five Feet Apart',
 'Captive State',
 '

In [6]:
# add Official Trailer for youtube search 
movies_trailer = [m + " Official Trailer" for m in encode_list]

In [8]:
# need to separate movie list into 20 to keep youtube requests small
split = [movies_trailer[x:x+20] for x in range(0, len(movies_trailer), 20)]
split

[['355 Official Trailer',
  'Moonfall Official Trailer',
  'Hannibal the Conqueror Official Trailer',
  'Unhinged Official Trailer',
  'Tenet Official Trailer',
  'Call of the Wild Official Trailer',
  'Army of the Dead Official Trailer',
  'Down Under Cover Official Trailer',
  'Rogue City Official Trailer',
  'Eli Official Trailer',
  'Reagan Official Trailer',
  'The Rhythm Section Official Trailer',
  'Midway Official Trailer',
  'Arctic Dogs Official Trailer',
  'Gisaengchung Official Trailer',
  'Ad Astra Official Trailer',
  'The Goldfinch Official Trailer',
  'PLAYMOBIL Official Trailer',
  'Blinded by the Light Official Trailer',
  'Once Upon a Timeâ\x80¦in Hollywood Official Trailer'],
 ['Crawl Official Trailer',
  'Spider-Man: Far From Home Official Trailer',
  'Anna Official Trailer',
  'Burn Your Maps Official Trailer',
  'Kursk Official Trailer',
  'Men in Black: International Official Trailer',
  'Shaft Official Trailer',
  'Late Night Official Trailer',
  'The Secret Li

In [10]:
# 12 lists from total 220, each 20 
m1 = split[0]
m2 = split[1]
m3 = split[2]
m4 = split[3]
m5 = split[4]
m6 = split[5]
m7 = split[6]
m8 = split[7]
m9 = split[8]
m10 = split[9]
m11= split[10]
m12 = split[11]

In [ ]:
base_url = "https://www.googleapis.com/youtube/v3/"
  
# new dictionary
d = {}

# try to retrieve videoID for movies in m1 first 
for movie in m1: 
    search_url = base_url + ("search?part=snippet&maxResults=1&q={0}&key={1}").format(movie, gkey)
    
    response = requests.get(search_url).json()
    # check response
    pprint(response)
    
    # add to key-value pair of movie and videoID to dictionary
    d[movie] = response["items"][0]["id"]["videoId"]

# create another (same) for loop for m2, m3..etc 

## only run this part if requests above work!

In [ ]:
# create dataframe from dictionary
df = pd.DataFrame.from_dict(list(d.items()))
df = df.rename(columns={0: "movie_trailer", 1: "video_id"})
df

In [ ]:
df["video_title"] = ""
df["channel_name"] = ""
df["channel_id"] = ""
df["comment_count"] = ""
df["view_count"] = ""
df["like_count"] = ""
df["date_published"] = ""

# for loop to retrieve view count, comment count, like count for each video
for index, row in df.iterrows():
    video = row["video_id"]
    index = index
    
    final_url = base_url + ("videos?part=snippet%2C+statistics&id={0}&key={1}").format(video, gkey)
    final_results = requests.get(final_url).json()
    vid_response = final_results["items"]
    
    for i in vid_response:
        t = i.get("snippet").get("title")
        c = i.get("snippet").get("channelTitle")
        d = i.get("snippet").get("channelId")
        m = i.get("snippet").get("publishedAt")

        comment_count = i.get("statistics").get("commentCount")
        view_count = i.get("statistics").get("viewCount")
        like_count = i.get("statistics").get("likeCount")
      
        df.loc[index, "video_title"] = t
        df.loc[index, "channel_name"] = c
        df.loc[index, "channel_id"] = d
        df.loc[index, "comment_count"] = comment_count
        df.loc[index, "view_count"] = view_count
        df.loc[index, "like_count"] = like_count
        df.loc[index, "date_published"] = m
df